<a href="https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/2_gym_wrappers_saving_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines3 Tutorial - Gym wrappers, saving and loading models

Github repo: https://github.com/araffin/rl-tutorial-jnrr19/tree/sb3/

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentation: https://stable-baselines3.readthedocs.io/en/master/

SB3-Contrib: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo


## Introduction

In this notebook, you will learn how to use *Gym Wrappers* which allow to do monitoring, normalization, limit the number of steps, feature augmentation, ...


You will also see the *loading* and *saving* functions, and how to read the outputted files for possible exporting.

## Install Dependencies and Stable Baselines3 Using Pip

In [59]:
# for autoformatting
# %load_ext jupyter_black

In [60]:
!pip install swig
!pip install "stable-baselines3[extra]>=2.0.0a4"

In [61]:
import gymnasium as gym
from stable_baselines3 import A2C, SAC, PPO, TD3

# Saving and loading

Saving and loading stable-baselines models is straightforward: you can directly call `.save()` and `.load()` on the models.

In [62]:
import os

# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

model = PPO("MlpPolicy", "Pendulum-v1", verbose=0).learn(8_000)
# The model will be saved under PPO_tutorial.zip
model.save(f"{save_dir}/PPO_tutorial")

# sample an observation from the environment
obs = model.env.observation_space.sample()

# Check prediction before saving
print("pre saved", model.predict(obs, deterministic=True))

del model  # delete trained model to demonstrate loading

loaded_model = PPO.load(f"{save_dir}/PPO_tutorial")
# Check that the prediction is the same after loading (for the same observation)
print("loaded", loaded_model.predict(obs, deterministic=True))

pre saved (array([0.28816497], dtype=float32), None)
loaded (array([0.28816497], dtype=float32), None)


Saving in stable-baselines is quite powerful, as you save the training hyperparameters, with the current weights. This means in practice, you can simply load a custom model, without redefining the parameters, and continue learning.

The loading function can also update the model's class variables when loading.

In [63]:
import os
from stable_baselines3.common.vec_env import DummyVecEnv

# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

model = A2C("MlpPolicy", "Pendulum-v1", verbose=0, gamma=0.9, n_steps=20).learn(8000)
# The model will be saved under A2C_tutorial.zip
model.save(f"{save_dir}/A2C_tutorial")

del model  # delete trained model to demonstrate loading

# load the model, and when loading set verbose to 1
loaded_model = A2C.load(f"{save_dir}/A2C_tutorial", verbose=1)

# show the save hyperparameters
print(f"loaded: gamma={loaded_model.gamma}, n_steps={loaded_model.n_steps}")

# as the environment is not serializable, we need to set a new instance of the environment
loaded_model.set_env(DummyVecEnv([lambda: gym.make("Pendulum-v1")]))
# and continue training
loaded_model.learn(8_000)

loaded: gamma=0.9, n_steps=20
------------------------------------
| time/                 |          |
|    fps                | 1756     |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.42    |
|    explained_variance | 0.0222   |
|    learning_rate      | 0.0007   |
|    n_updates          | 499      |
|    policy_loss        | -15.8    |
|    std                | 0.998    |
|    value_loss         | 417      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 1674     |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -1.43    |
|    explained_variance | 0.000403 |
|    learning_rate      | 0.0007   |
|    n_updates          | 599      |
|    pol

# Gym and VecEnv wrappers

## Anatomy of a gym wrapper

A gym wrapper follows the [gym](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html) interface: it has a `reset()` and `step()` method.

Because a wrapper is *around* an environment, we can access it with `self.env`, this allow to easily interact with it without modifying the original env.
There are many wrappers that have been predefined, for a complete list refer to [gym documentation](https://gymnasium.farama.org/api/wrappers/)

In [64]:
class CustomWrapper(gym.Wrapper):
    """
    :param env: (gym.Env) Gym environment that will be wrapped
    """

    def __init__(self, env):
        # Call the parent constructor, so we can access self.env later
        super().__init__(env)

    def reset(self, **kwargs):
        """
        Reset the environment
        """
        obs, info = self.env.reset(**kwargs)

        return obs, info

    def step(self, action):
        """
        :param action: ([float] or int) Action taken by the agent
        :return: (np.ndarray, float, bool, bool, dict) observation, reward, is this a final state (episode finished),
        is the max number of steps reached (episode finished artificially), additional informations
        """
        obs, reward, terminated, truncated, info = self.env.step(action)
        return obs, reward, terminated, truncated, info

## First example: limit the episode length

One practical use case of a wrapper is when you want to limit the number of steps by episode, for that you will need to overwrite the `done` signal when the limit is reached. It is also a good practice to pass that information in the `info` dictionary.

In [65]:
class TimeLimitWrapper(gym.Wrapper):
    """
    :param env: (gym.Env) Gym environment that will be wrapped
    :param max_steps: (int) Max number of steps per episode
    """

    def __init__(self, env, max_steps=100):
        # Call the parent constructor, so we can access self.env later
        super(TimeLimitWrapper, self).__init__(env)
        self.max_steps = max_steps
        # Counter of steps per episode
        self.current_step = 0

    def reset(self, **kwargs):
        """
        Reset the environment
        """
        # Reset the counter
        self.current_step = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        """
        :param action: ([float] or int) Action taken by the agent
        :return: (np.ndarray, float, bool, bool, dict) observation, reward, is the episode over?, additional informations
        """
        self.current_step += 1
        obs, reward, terminated, truncated, info = self.env.step(action)
        # Overwrite the truncation signal when when the number of steps reaches the maximum
        if self.current_step >= self.max_steps:
            truncated = True
        return obs, reward, terminated, truncated, info

#### Test the wrapper

In [66]:
from gymnasium.envs.classic_control.pendulum import PendulumEnv

# Here we create the environment directly because gym.make() already wrap the environment in a TimeLimit wrapper otherwise
env = PendulumEnv()
# Wrap the environment
env = TimeLimitWrapper(env, max_steps=100)

In [67]:
obs, _ = env.reset()
done = False
n_steps = 0
while not done:
    # Take random actions
    random_action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(random_action)
    done = terminated or truncated
    n_steps += 1

print(n_steps, info)

100 {}


In practice, `gym` already have a wrapper for that named `TimeLimit` (`gym.wrappers.TimeLimit`) that is used by most environments.

## Second example: normalize actions

It is usually a good idea to normalize observations and actions before giving it to the agent, this prevents this [hard to debug issue](https://github.com/hill-a/stable-baselines/issues/473).

In this example, we are going to normalize the action space of *Pendulum-v1* so it lies in [-1, 1] instead of [-2, 2].

Note: here we are dealing with continuous actions, hence the `gym.Box` space

In [68]:
import numpy as np


class NormalizeActionWrapper(gym.Wrapper):
    """
    :param env: (gym.Env) Gym environment that will be wrapped
    """

    def __init__(self, env):
        # Retrieve the action space
        action_space = env.action_space
        assert isinstance(
            action_space, gym.spaces.Box
        ), "This wrapper only works with continuous action space (spaces.Box)"
        # Retrieve the max/min values
        self.low, self.high = action_space.low, action_space.high

        # We modify the action space, so all actions will lie in [-1, 1]
        env.action_space = gym.spaces.Box(
            low=-1, high=1, shape=action_space.shape, dtype=np.float32
        )

        # Call the parent constructor, so we can access self.env later
        super(NormalizeActionWrapper, self).__init__(env)

    def rescale_action(self, scaled_action):
        """
        Rescale the action from [-1, 1] to [low, high]
        (no need for symmetric action space)
        :param scaled_action: (np.ndarray)
        :return: (np.ndarray)
        """
        return self.low + (0.5 * (scaled_action + 1.0) * (self.high - self.low))

    def reset(self, **kwargs):
        """
        Reset the environment
        """
        return self.env.reset(**kwargs)

    def step(self, action):
        """
        :param action: ([float] or int) Action taken by the agent
        :return: (np.ndarray, float,bool, bool, dict) observation, reward, final state? truncated?, additional informations
        """
        # Rescale action from [-1, 1] to original [low, high] interval
        rescaled_action = self.rescale_action(action)
        obs, reward, terminated, truncated, info = self.env.step(rescaled_action)
        return obs, reward, terminated, truncated, info

#### Test before rescaling actions

In [69]:
original_env = gym.make("Pendulum-v1")

print(original_env.action_space.low)
for _ in range(10):
    print(original_env.action_space.sample())

[-2.]
[-0.8195075]
[-0.63990194]
[1.2563243]
[-0.4956313]
[-0.2720555]
[-1.7006481]
[1.6226037]
[0.40229142]
[-1.945991]
[-0.10842818]


#### Test the NormalizeAction wrapper

In [70]:
env = NormalizeActionWrapper(gym.make("Pendulum-v1"))

print(env.action_space.low)

for _ in range(10):
    print(env.action_space.sample())

[-1.]
[0.9283196]
[0.7437544]
[0.98674595]
[-0.691826]
[0.21223567]
[0.7195945]
[0.82176584]
[-0.5907611]
[0.6986454]
[0.3023669]


#### Test with a RL algorithm

We are going to use the Monitor wrapper of stable baselines, which allow to monitor training stats (mean episode reward, mean episode length)

In [71]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

In [72]:
env = Monitor(gym.make("Pendulum-v1"))
env = DummyVecEnv([lambda: env])

In [73]:
model = A2C("MlpPolicy", env, verbose=1).learn(int(1000))

Using cpu device
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.31e+03 |
| time/                 |           |
|    fps                | 997       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.42     |
|    explained_variance | -0.0517   |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -17.9     |
|    std                | 0.999     |
|    value_loss         | 288       |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.44e+03 |
| time/                 |           |
|    fps                | 1017      |
|    iterations         | 200       |
|    time_elapsed       | 0      

With the action wrapper

In [74]:
normalized_env = Monitor(gym.make("Pendulum-v1"))
# Note that we can use multiple wrappers
normalized_env = NormalizeActionWrapper(normalized_env)
normalized_env = DummyVecEnv([lambda: normalized_env])

In [75]:
model_2 = A2C("MlpPolicy", normalized_env, verbose=1).learn(int(1000))

Using cpu device
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.53e+03 |
| time/                 |           |
|    fps                | 1027      |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.44     |
|    explained_variance | 0.00199   |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -42.4     |
|    std                | 1.02      |
|    value_loss         | 659       |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.61e+03 |
| time/                 |           |
|    fps                | 1009      |
|    iterations         | 200       |
|    time_elapsed       | 0      

## Additional wrappers: VecEnvWrappers

In the same vein as gym wrappers, stable baselines provide wrappers for `VecEnv`. Among the different wrappers that exist (and you can create your own), you should know: 

- VecNormalize: it computes a running mean and standard deviation to normalize observation and returns
- VecFrameStack: it stacks several consecutive observations (useful to integrate time in the observation, e.g. successive frame of an atari game)

More info in the [documentation](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#wrappers)

Note: when using `VecNormalize` wrapper, you must save the running mean and std along with the model, otherwise you will not get proper results when loading the agent again. If you use the [rl zoo](https://github.com/DLR-RM/rl-baselines3-zoo), this is done automatically

In [76]:
from stable_baselines3.common.vec_env import VecNormalize, VecFrameStack

env = DummyVecEnv([lambda: gym.make("Pendulum-v1")])
normalized_vec_env = VecNormalize(env)

In [77]:
obs = normalized_vec_env.reset()
for _ in range(10):
    action = [normalized_vec_env.action_space.sample()]
    obs, reward, _, _ = normalized_vec_env.step(action)
    print(obs, reward)

[[-0.4794123 -0.8721114 -0.9993201]] [-10.]
[[-1.1590378 -1.3017055 -1.2784799]] [-2.0087214]
[[-1.5032765 -1.4998912 -1.3801733]] [-1.529878]
[[-1.7093207 -1.6287401 -1.5051893]] [-1.3563831]
[[-1.8425254 -1.6921432 -1.5175936]] [-1.2920336]
[[-1.9236135 -1.6974399 -1.4469849]] [-1.1950265]
[[-2.0309143 -1.6955304 -1.6608964]] [-1.0660247]
[[-2.118985  -1.6280692 -1.7650849]] [-1.0952913]
[[-2.1563175 -1.4352325 -1.7285138]] [-1.0946115]
[[-2.1556003 -1.0369928 -1.7234594]] [-1.0302227]


## Exercise: code you own monitor wrapper

Now that you know how does a wrapper work and what you can do with it, it's time to experiment.

The goal here is to create a wrapper that will monitor the training progress, storing both the episode reward (sum of reward for one episode) and episode length (number of steps in for the last episode).

You will return those values using the `info` dict after each end of episode.

In [78]:
class MyMonitorWrapper(gym.Wrapper):
    """
    :param env: (gym.Env) Gym environment that will be wrapped
    """

    def __init__(self, env):
        # Call the parent constructor, so we can access self.env later
        super().__init__(env)
        # === YOUR CODE HERE ===#
        # Initialize the variables that will be used
        # to store the episode length and episode reward
        rewards_sum = 0
        episode_length = 0

        # ====================== #

    def reset(self, **kwargs):
        """
        Reset the environment
        """
        obs = self.env.reset(**kwargs)
        # === YOUR CODE HERE ===#
        # Reset the variables
        self.rewards_sum = self.episode_length = 0
        # ====================== #
        return obs

    def step(self, action):
        """
        :param action: ([float] or int) Action taken by the agent
        :return: (np.ndarray, float, bool, bool, dict)
            observation, reward, is the episode over?, is the episode truncated?, additional information
        """
        obs, reward, terminated, truncated, info = self.env.step(action)
        # === YOUR CODE HERE ===#
        # Update the current episode reward and episode length
        self.rewards_sum += reward
        self.episode_length += 1
        # ====================== #

        if terminated or truncated:
            # === YOUR CODE HERE ===#
            # Store the episode length and episode reward in the info dict
            info = {"episode": {"r": self.rewards_sum, "l": self.episode_length}}

            pass

            # ====================== #
        return obs, reward, terminated, truncated, info

#### Test your wrapper

In [79]:
# To use LunarLander, you need to install box2d box2d-kengz (pip) and swig (apt-get)
!pip install box2d-py

In [80]:
env = gym.make("LunarLander-v3")
# === YOUR CODE HERE ===#
# Wrap the environment
env = MyMonitorWrapper(env)
# Reset the environment
env.reset()
# Take random actions in the environment and check
# that it returns the correct values after the end of each episode
done = False

for _ in range (1000):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    if done:
        print(info)
        obs = env.reset()
# ====================== #

{'episode': {'r': -182.6984657952276, 'l': 64}}
{'episode': {'r': -240.3060261288511, 'l': 109}}
{'episode': {'r': -112.99877033824873, 'l': 84}}
{'episode': {'r': -296.5366683585005, 'l': 104}}
{'episode': {'r': -117.54218847637739, 'l': 88}}
{'episode': {'r': -197.86532151460494, 'l': 71}}
{'episode': {'r': -243.89429083654258, 'l': 110}}
{'episode': {'r': -142.7933120077028, 'l': 87}}
{'episode': {'r': -125.23181377643166, 'l': 113}}
{'episode': {'r': -91.63621020404483, 'l': 79}}
{'episode': {'r': -26.781923858668108, 'l': 65}}


 # Conclusion
 
 In this notebook, we have seen:
 - how to easily save and load a model
 - what is wrapper and what we can do with it
 - how to create your own wrapper

## Wrapper Bonus: changing the observation space: a wrapper for episode of fixed length

In [85]:
# from gymna.wrappers import TimeLimit
from gymnasium.wrappers import TimeLimit

class TimeFeatureWrapper(gym.Wrapper):
    """
    Add remaining time to observation space for fixed length episodes.
    See https://arxiv.org/abs/1712.00378 and https://github.com/aravindr93/mjrl/issues/13.

    :param env: (gym.Env)
    :param max_steps: (int) Max number of steps of an episode
        if it is not wrapped in a TimeLimit object.
    :param test_mode: (bool) In test mode, the time feature is constant,
        equal to zero. This allow to check that the agent did not overfit this feature,
        learning a deterministic pre-defined sequence of actions.
    """

    def __init__(self, env, max_steps=1000, test_mode=False):
        assert isinstance(env.observation_space, gym.spaces.Box)
        # Add a time feature to the observation
        low, high = env.observation_space.low, env.observation_space.high
        low, high = np.concatenate((low, [0])), np.concatenate((high, [1.0]))
        env.observation_space = gym.spaces.Box(low=low, high=high, dtype=np.float32)

        super().__init__(env)

        if isinstance(env, TimeLimit):
            self._max_steps = env._max_episode_steps
        else:
            self._max_steps = max_steps
        self._current_step = 0
        self._test_mode = test_mode

    def reset(self, **kwargs):
        self._current_step = 0
        obs, info = self.env.reset(**kwargs)
        return self._get_obs(obs), info

    def step(self, action):
        self._current_step += 1
        obs, reward, terminated, truncated, info = self.env.step(action)
        return self._get_obs(obs), reward, terminated, truncated, info

    def _get_obs(self, obs):
        """
        Concatenate the time feature to the current observation.

        :param obs: (np.ndarray)
        :return: (np.ndarray)
        """
        # Remaining time is more general
        time_feature = 1 - (self._current_step / self._max_steps)
        if self._test_mode:
            time_feature = 1.0
        # Optionally: concatenate [time_feature, time_feature ** 2]
        return np.concatenate((obs, [time_feature]))

In [86]:
env = gym.make("CartPole-v1")

wrapped_env = TimeFeatureWrapper(env, max_steps=500)

obs, info = wrapped_env.reset()
for _ in range(10):
    action = wrapped_env.action_space.sample()
    obs, reward, terminated, truncated, info = wrapped_env.step(action)
    print(f"Observação: {obs}, Recompensa: {reward}, Terminado: {terminated}, Truncado: {truncated}")
    if terminated or truncated:
        obs, info = wrapped_env.reset()

Observação: [ 0.03791334  0.21629137 -0.04922095 -0.31341743  0.998     ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.04223917  0.41207868 -0.05548929 -0.62120783  0.996     ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.05048075  0.21777375 -0.06791345 -0.34650451  0.994     ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.05483622  0.02368024 -0.07484354 -0.07598659  0.992     ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.05530982 -0.17029342 -0.07636327  0.19217589  0.99      ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.05190396  0.02583317 -0.07251976 -0.12358564  0.988     ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.05242062 -0.16817893 -0.07499147  0.14536548  0.986     ], Recompensa: 1.0, Terminado: False, Truncado: False
Observação: [ 0.04905704  0.02793238 -0.07208416 -0.17000157  0.984     ], Recompensa: 1.0, Terminado: False, Truncado

## Going further - Saving format 

The format for saving and loading models is a zip-archived JSON dump and NumPy zip archive of the arrays:
```
saved_model.zip/
├── data              JSON file of class-parameters (dictionary)
├── parameter_list    JSON file of model parameters and their ordering (list)
├── parameters        Bytes from numpy.savez (a zip file of the numpy arrays). ...
    ├── ...           Being a zip-archive itself, this object can also be opened ...
        ├── ...       as a zip-archive and browsed.
```

## Save and find 

In [87]:
# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

model = PPO("MlpPolicy", "Pendulum-v1", verbose=0).learn(8000)
model.save(save_dir + "/PPO_tutorial")

In [88]:
!ls /tmp/gym/PPO_tutorial*

/tmp/gym/PPO_tutorial.zip


In [89]:
import zipfile

archive = zipfile.ZipFile("/tmp/gym/PPO_tutorial.zip", "r")
for f in archive.filelist:
    print(f.filename)

data
pytorch_variables.pth
policy.pth
policy.optimizer.pth
_stable_baselines3_version
system_info.txt


## Exporting saved models

And finally some further reading for those who want to export to tensorflowJS or Java.

https://stable-baselines3.readthedocs.io/en/master/guide/export.html